## 1. Review the course programming code

In [1]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [2]:
from collections import defaultdict
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i+1] = p

solution = {}

#方法1：利用字典max_price记录所有<=n的最优切割方法
def cut(n):
    max_price = {}
    
    for i in range(1, n+1):
        max_price[i], split = max([(price[i], 0)] + [(max_price[j] + max_price[i-j], j) for j in range(1, i)],
                                       key = lambda x:x[0])
        solution[i] = (i - split, split)
        
    return max_price[n], parse(n)

def parse(n):
    left, right = solution[n]
    if right == 0:
        return [left]
    else:
        return parse(left) + parse(right)
    

In [3]:
cut(20)

(60, [11, 6, 3])

In [4]:
from functools import wraps

def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

In [5]:
#方法2：利用装饰器@memo的方法记录缓存
@memo
def cut(n):
    max_price, split = max([(price[n], 0)] + [(cut(j) + cut(n-j), j) for j in range(1, n)],
                                       key = lambda x:x[0])
    solution[n] = (n - split, split)    
    return max_price

def parse(n):
    left, right = solution[n]
    if right == 0:
        return [left]
    else:
        return parse(left) + parse(right)

In [6]:
cut(20)

60

In [7]:
parse(20)

[11, 6, 3]

## 2. Complete the Edit-Distance Problem Solution, by which we could get the detailed transformer procedure of two string X and Y.

In [8]:
from functools import lru_cache

operations = {}


@lru_cache(maxsize=2**10)
def edit_distance(s1, s2):
    
    if len(s1) == 0:
        return len(s2)
    if len(s2) == 0:
        return len(s1)
    
    if s1[-1] == s2[-1]:
        dist, op = min((edit_distance(s1[:-1], s2[:-1]),'Do Nothing'),
                       (edit_distance(s1[:-1], s2) + 1, 'DEL {}'.format(s1[-1])),
                       (edit_distance(s1, s2[:-1]) + 1, 'ADD {}'.format(s2[-1])), 
                        key = lambda x:x[0])
        operations[(s1, s2)] = op
        return dist
    else:
        dist, op = min((edit_distance(s1[:-1], s2[:-1]) + 1, 'SUB {}->{}'.format(s1[-1], s2[-1])),
                       (edit_distance(s1[:-1], s2) + 1, 'DEL {}'.format(s1[-1])),
                       (edit_distance(s1, s2[:-1]) + 1, 'ADD {}'.format(s2[-1])),
                        key = lambda x:x[0])
        operations[(s1, s2)] = op
        return dist
        
def parse_operations(s1, s2):
    if len(s1) == 0 and len(s2) == 0:
        return
    elif len(s1) == 0:
        print('{}->{}: ADD {}'.format(s1, s2, s2))
        return
    elif len(s2) == 0:
        print('{}->{}: DEL {}'.format(s1, s2, s1))
        return
        
    op = operations[s1, s2]
    print('{}->{}: {}'.format(s1, s2, op))
    if 'Do Nothing' in op:
        return parse_operations(s1[:-1], s2[:-1])
    elif 'SUB' in op:
        return parse_operations(s1[:-1], s2[:-1])
    elif 'ADD' in op:
        return parse_operations(s1, s2[:-1])
    elif 'DEL' in op:
        return parse_operations(s1[:-1], s2)

In [9]:
s1 = 'ABCDECG'
s2 = 'ABCCEF'
print(edit_distance(s1, s2))
parse_operations(s1, s2)

3
ABCDECG->ABCCEF: SUB G->F
ABCDEC->ABCCE: DEL C
ABCDE->ABCCE: Do Nothing
ABCD->ABCC: SUB D->C
ABC->ABC: Do Nothing
AB->AB: Do Nothing
A->A: Do Nothing


## 3. Complete the Pinyin auto-correction problem

In [10]:
chinese_dataset = '../week1/article_9k.txt'
CHINESE_CHARACTERS = open(chinese_dataset).read()

In [11]:
CHINESE_CHARACTERS[:100]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MI'

In [12]:
import re
def token(string):
    return re.findall('[\u4e00-\u9fa5]+', string)

In [13]:
token(CHINESE_CHARACTERS[:10])

['此外自本周', '月', '日']

In [14]:
import pinyin
pinyin.get(''.join(token(CHINESE_CHARACTERS[:100])), format='strip', delimiter=' ')

'ci wai zi ben zhou yue ri qi chu xiao mi shou ji deng kuan ji xing wai qi yu ji xing yi zan ting geng xin fa bu han kai fa ban ti yan ban nei ce wen ding ban zan bu shou ying xiang yi que bao gong cheng shi ke yi ji zhong quan bu jing li jin xing xi tong you hua gong zuo you ren cai ce zhe ye shi jiang jing li zhu yao yong dao de yan fa zhi zhong'

In [15]:
CHINESE_PINYIN = pinyin.get(''.join(token(CHINESE_CHARACTERS)), format='strip', delimiter=' ')

In [16]:
CHINESE_PINYIN[:100]

'ci wai zi ben zhou yue ri qi chu xiao mi shou ji deng kuan ji xing wai qi yu ji xing yi zan ting gen'

In [17]:
def token_pinyin(string):
    return re.findall('[a-z]+', string.lower())

In [18]:
token_pinyin(CHINESE_PINYIN[:10])

['ci', 'wai', 'zi']

In [19]:
from collections import Counter, defaultdict
counter = Counter(token_pinyin(CHINESE_PINYIN))

In [20]:
counter.most_common(10)

[('shi', 860634),
 ('de', 809887),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441),
 ('zai', 334106)]

In [21]:
len(counter)

397

In [22]:
def edit0(word):
    return {word}

# def splits(word):
#     return [word[:i], word[i:] for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'
def edit1(word):
#     pairs = splits(word)
    deletes = [word[:i] + word[i+1:] for i in range(len(word))]
    inserts = [word[:i] + c + word[i:] for i in range(len(word)+1) for c in alphabet]
    replaces = [word[:i] + c + word[i+1:] for i in range(len(word)) for c in alphabet]
    transposes = [word[:i] + word[i+1] + word[i] + word[i+2:] for i in range(len(word)-1)]
    return set(deletes + inserts + replaces + transposes)

def edit2(word):
    return {e2 for e1 in edit1(word) for e2 in edit1(e1)}

In [23]:
def known(words):
    return {w for w in words if w in counter}

In [24]:
known(edit1('ping'))

{'bing',
 'ding',
 'jing',
 'ling',
 'ming',
 'ning',
 'pang',
 'peng',
 'pin',
 'ping',
 'qing',
 'ting',
 'xing',
 'ying'}

In [25]:
def correct(word):
    candidates = (known(edit0(word)) or known(edit1(word)) or known(edit2(word)))
    return max(candidates, key=counter.get)

In [26]:
correct('pign')

'ping'

In [27]:
correct('yign')

'ying'

In [28]:
correct('yngi')

'yi'

In [29]:
correct('ygi')

'yi'

In [30]:
def correct_sentences_pinyin(text_pinyin):
    words = text_pinyin.split(' ')
    return ' '.join([correct(word) for word in words])

In [31]:
correct_sentences_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

In [32]:
correct_sentences_pinyin('wo xiang shagn qinng hua da xeu')

'wo xiang shang qing hua da xue'

In [33]:
correct_sentences_pinyin('zhe jiang gogn ye da xue')

'zhe jiang gong ye da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

In [34]:
word_list = CHINESE_PINYIN.split(' ')
word_list_2 = [''.join(word_list[i:i+2]) for i in range(len(word_list)-1)]
counter_2 = Counter(word_list_2)

In [35]:
counter_2.most_common(10)

[('yueri', 165283),
 ('xinhua', 151830),
 ('huashe', 145990),
 ('zhongguo', 87894),
 ('waidai', 83330),
 ('nianyue', 78062),
 ('jizhe', 65398),
 ('erxian', 62431),
 ('daier', 61784),
 ('zhaopian', 52348)]

In [36]:
# def get_prob2(word1, word2):
#     if word1 + word2 in word_list_2:
#         return counter_2[word1+word2] / sum(counter_2.values())
#     else:
#         return 0

#在分隔拼音的时候，只考虑编辑距离为0和编辑距离为1的错误，使得分隔较为准确。
def correct1(word):
    candidates = (known(edit0(word)) or known(edit1(word)))
    return max(candidates, key=counter.get)

def get_prob1(word):
    delta = 1e-5
    #delta为惩罚因子，避免将类似'wode'的组合拼音识别为需要correct的一个单词'wo'而非'wo'+'de'。
    if word in word_list:
        return counter[word] / sum(counter.values())
    else:
        try:
            correct_word = correct1(word)
            return delta * counter[correct_word] / sum(counter.values())
        except:
            return 0
    
splits = {}


@lru_cache(maxsize=2**10)
def split_pinyin(words):
    max_prob, split = max([(get_prob1(words),0)]
                      + [((split_pinyin(words[:i]) * split_pinyin(words[i:])), i) for i in range(1, len(words))],
                       key = lambda x:x[0])
    splits[words] = split
    return max_prob

def parse(words):
    max_prob = split_pinyin(words)
    i = splits[words]
    if i == 0:
        return [words]
    left, right = words[:i], words[i:]
    return parse(left) + parse(right)

In [37]:
string = 'qinnghuadaxeu'

In [38]:
split_string = ' '.join(parse(string))
correct_sentences_pinyin(split_string)

'qing hua da xue'

In [39]:
split_string = ' '.join(parse('zhesihyigecesho'))
correct_sentences_pinyin(split_string)

'zhe shi yi ge ce shi'

In [40]:
split_string = ' '.join(parse('pniyinshurufw'))
correct_sentences_pinyin(split_string)

'pin yin shu ru fa'

当前拼音纠错的模型只考虑单个拼音是否正确，还不能讲'shurifa'之类的错误更正为'shurufa'。后续如果在correct_sentence_pinyin中，加入n-gram语言模型，可能可以解决这种问题。另外，分隔拼音的split_pinyin函数，目前运行时间还是过长，还有改进的空间。